In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time # for sleep time
import csv
import re
import functools
import datetime #for csv file title

driver = webdriver.Chrome(r'C:\Users\priya\chromedriver.exe')

driver.get('https://www.airbnb.com/s/Orlando--FL--United-States/homes?refinement_paths%5B%5D=%2Fhomes&query=Orlando%2C%20FL%2C%20United%20States&adults=3&children=0&infants=0&guests=0&toddlers=0&place_id=ChIJd7zN_thz54gRnr-lPAaywwo&search_type=SECTION_NAVIGATION&allow_override%5B%5D=&s_tag=R68fdmv6')

now = datetime.datetime.now() #for csv file title

#For csv file names
def reduce_concat(x, sep=""):
    return functools.reduce(lambda x, y: str(x) + sep + str(y), x)

def paste(*lists, sep=" ", collapse=None):
    result = map(lambda x: reduce_concat(x, sep=sep), zip(*lists))
    if collapse is not None:
        return reduce_concat(result, sep=collapse)
    return list(result)

#Open CSV Writer
filename = paste(['airBnb_Orlando'], [now.year], ['-'], [now.month], ['-'], [now.day],['.csv'], sep='')
filename = ''.join(filename)
csv_file = open(filename, 'w', encoding='utf-8',newline = '')
writer = csv.writer(csv_file)

index = 1

house_all_links = []

while True:
    
    if index > 17:
        break
    try:
        print("Getting list of links from Page " + str(index))
        index = index + 1
        #driver.maximize_window()
        time.sleep(6) # give new page time to load
                
        house_list = (driver.find_elements_by_xpath('//a[@class="_1ol0z3h"]'))

        house_links = [house.get_attribute('href') for house in house_list ] 
        
        house_all_links.extend(house_links)
  
        # Locate the next button on the page.
        wait_button = WebDriverWait(driver, 10)

        next_button = wait_button.until(EC.element_to_be_clickable((By.XPATH,'//li[@class="_r4n1gzb"]')))
        
        next_button.click()

        
    except Exception as e:
        print(e)
        continue
        
print(house_all_links)

for url in house_all_links:
   
    # Initialize an empty dictionary for each review
    airbnb_dict = {}
    driver.get(url)

    print(url)
    try:
        time.sleep(1.5)
        try:
            house_title = driver.find_element_by_xpath('//h1/span[@class="_18hrqvin"]').text
        except:
            print("issue in house title")
            house_title = None

        try:
            house_type =  driver.find_element_by_xpath('//div[@class="_1p3joamp"]').text
        except:
            print("issue in house type")
            house_type = None
            
        time.sleep(3)    

        try:
            
            price = driver.find_element_by_xpath('//span/span[@class="_doc79r"]').text
        except:
            print("issue in price")
            price = None        

        try:
            user = driver.find_element_by_xpath('//div[@class="_8b6uza1"]').text
        except:
            print("issue in user")
            user = None  

        try:
            no_of_ratings = driver.find_element_by_xpath('//div[@class="_vy3ibx"]/h2/span').text

        except:
            print("issue in no_of_ratings")
            no_of_ratings = None  
        
        time.sleep(3.5)    
        try:
            rating = driver.find_element_by_xpath('//div[@class="_l0ao8q"]//div[@itemprop="ratingValue"]').get_attribute('content')

        except:
            print("issue in rating")
            rating = None

        time.sleep(2)        
        try:
            max_occupancy = driver.find_elements_by_xpath('//div/div[@class="_36rlri"]/div[@class="_czm8crp"]')

            house_detail = [detail.text for detail in max_occupancy ]

        except:
            print("Issue in occupncy")
            house_detail = None

        airbnb_dict['house_title'] = house_title
        airbnb_dict['house_type'] = house_type
        airbnb_dict['price'] = price
        airbnb_dict['user'] = user
        airbnb_dict['no_of_reviews'] = no_of_ratings
        airbnb_dict['rating'] = rating
        airbnb_dict['house_detail'] = house_detail

        print("writing")
        writer.writerow(airbnb_dict.values())

        print(i)
        print(house_title)
        print(house_type)
        print(price)
        print(user)
        print(no_of_ratings)
        print(rating)
        print(house_detail)

    except Exception as e:
        print(e)
        continue
        
csv_file.close()
driver.close()